In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import duckdb
from tqdm import tnrange, tqdm_notebook as tqdm

In [4]:
conn = duckdb.connect('../data/expedia.duckdb')
c = conn.cursor()

In [ ]:
c.execute("""
CREATE TABLE train(

date_time text,
site_name integer,
posa_continent integer,
user_location_country integer,
user_location_region integer,
user_location_city integer,
orig_destination_distance integer,
user_id integer,
is_mobile integer,
is_package integer,
channel integer,
srch_ci date,
srch_co date,
srch_adults_cnt integer,
srch_children_cnt integer,
srch_rm_cnt integer,
srch_destination_id integer,
srch_destination_type_id integer,
is_booking integer,
cnt integer,
hotel_continent integer,
hotel_country integer,
hotel_market integer,
hotel_cluster integer)
""")

In [5]:
#c.execute("copy train from '../data/train.csv' WITH (HEADER 1, DELIMITER ',')")

In [13]:
cnt_user_ids = c.execute("SELECT COUNT(DISTINCT user_id) FROM train  WHERE is_booking = 0").fetchdf()

In [14]:
cnt_user_ids

,count(user_id)
0,0


In [10]:
c.execute("DELETE FROM train WHERE is_booking != 1")

There are approximately 8 million unique user ID's making a booking. From online discussions, it transpires that travel agents use Expedia too. It would be useful to remove these entries. Others have decided that a cut off of >20 bookings in the data indicates a travel agent. I'll go with this here.

In [15]:
user_ids = c.execute("SELECT user_id FROM (SELECT user_id, COUNT(*) AS counter FROM train GROUP by user_id) AS tbl WHERE counter >20").fetchdf()

In [16]:
len(user_ids)

14686

In [17]:
user_ids.head()

,user_id
0,114307
1,386791
2,1170970
3,124725
4,304711


In [18]:
c.execute("""
DROP TABLE user_ids
""")

In [19]:
c.execute("""
CREATE TABLE user_ids(user_id integer)
""")

In [20]:
for d in tqdm(user_ids.user_id):
    sql = "INSERT into user_ids(user_id) VALUES ({})".format(d)
    c.execute(sql)

/home/stephen/minimamba/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [25]:
c.execute("DROP TABLE train_final")

In [23]:
c.execute("""
CREATE TABLE train_final(

date_time text,
site_name integer,
posa_continent integer,
user_location_country integer,
user_location_region integer,
user_location_city integer,
orig_destination_distance integer,
user_id integer,
is_mobile integer,
is_package integer,
channel integer,
srch_ci date,
srch_co date,
srch_adults_cnt integer,
srch_children_cnt integer,
srch_rm_cnt integer,
srch_destination_id integer,
srch_destination_type_id integer,
is_booking integer,
cnt integer,
hotel_continent integer,
hotel_country integer,
hotel_market integer,
hotel_cluster integer)
""")

DatabaseError: Catalog: Table or view with name "train_final" already exists!

Filter train table on the ids in user_ids. export final data set to csv for machine learning

In [27]:
train_final = c.execute("SELECT * FROM train WHERE EXISTS(SELECT * FROM user_ids WHERE train.user_id=user_ids.user_id)").fetchdf()

DatabaseError: IO: Could not read sufficient bytes from file "../data/expedia.duckdb"

In [ ]:
train_final.to_csv('../data/train_final.csv')

In [ ]:
!ls -lh ../data

In [ ]:
c.close()